In [ ]:
!wget https://dl.fbaipublicfiles.com/torchbiggraph/wikidata_translation_v1.tsv.gz -O wikidata_translation_v1.tsv.gz


--2024-10-16 12:12:29--  https://dl.fbaipublicfiles.com/torchbiggraph/wikidata_translation_v1.tsv.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.176.19, 18.238.176.44, 18.238.176.115, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.176.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38830884950 (36G) [application/gzip]
Saving to: ‘wikidata_translation_v1.tsv.gz’

wikidata_translatio 100%[===================>]  36.16G  54.5MB/s    in 9m 50s  

2024-10-16 12:22:20 (62.7 MB/s) - ‘wikidata_translation_v1.tsv.gz’ saved [38830884950/38830884950]



Download Wikimapper

In [ ]:
!wget https://public.ukp.informatik.tu-darmstadt.de/wikimapper/index_enwiki-20190420.db -O index_enwiki-20190420.db



--2024-10-16 12:43:42--  https://public.ukp.informatik.tu-darmstadt.de/wikimapper/index_enwiki-20190420.db
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1607753728 (1.5G) [application/octet-stream]
Saving to: ‘index_enwiki-20190420.db’

index_enwiki-201904 100%[===================>]   1.50G  9.08MB/s    in 3m 11s  

2024-10-16 12:46:54 (8.02 MB/s) - ‘index_enwiki-20190420.db’ saved [1607753728/1607753728]



In [ ]:
!wget https://public.ukp.informatik.tu-darmstadt.de/wikimapper/index_dewiki-20190420.db -O index_dewiki-20190420.db

!wget https://public.ukp.informatik.tu-darmstadt.de/wikimapper/index_eswiki-20190420.db -O index_eswiki-20190420.db

!wget https://public.ukp.informatik.tu-darmstadt.de/wikimapper/index_frwiki-20190420.db -O index_frwiki-20190420.db

!wget https://public.ukp.informatik.tu-darmstadt.de/wikimapper/index_jawiki-20190420.db -O index_jawiki-20190420.db

--2024-10-16 16:57:15--  https://public.ukp.informatik.tu-darmstadt.de/wikimapper/index_dewiki-20190420.db
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 382685184 (365M) [application/octet-stream]
Saving to: ‘index_dewiki-20190420.db’

index_dewiki-201904 100%[===================>] 364.96M  4.90MB/s    in 98s     

2024-10-16 16:58:54 (3.72 MB/s) - ‘index_dewiki-20190420.db’ saved [382685184/382685184]

--2024-10-16 16:58:54--  https://public.ukp.informatik.tu-darmstadt.de/wikimapper/index_eswiki-20190420.db
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request 

In [ ]:
!pip install wikimapper

  Preparing metadata (setup.py) ... done
  Created wheel for wikimapper: filename=wikimapper-0.2.0-py3-none-any.whl size=16136 sha256=08a54c4eac02171f4780333e92fd97080e6fc14ee69c53bd501cfe230b38a588
  Stored in directory: /root/.cache/pip/wheels/90/ce/b3/fdf588a8581836f956a4616e6e16bc708e6dbb1f4eabb58977
Successfully built wikimapper


Mapping author to Wikidata ID

In [ ]:
import pandas as pd
from wikimapper import WikiMapper
import pickle

# Load the WikiMapper indices (use the correct paths)
en_mapper = WikiMapper("/content/index_enwiki-20190420.db")  # English Wikipedia index
de_mapper = WikiMapper('/content/index_dewiki-20190420.db')  # title language is defined in index file
es_mapper = WikiMapper('/content/index_eswiki-20190420.db')
ja_mapper = WikiMapper('/content/index_jawiki-20190420.db')
fr_mapper = WikiMapper('/content/index_frwiki-20190420.db')



In [ ]:
# Example usage
author_list = ["Edward Lear", "James Patterson", "Jane Austen"]
author_qids = {author: get_qid_for_author(author, en_mapper) for author in author_list}

print(author_qids)


{'Edward Lear': 'Q309759', 'James Patterson': 'Q311671', 'Jane Austen': 'Q36322'}


In [ ]:
def clean_author_name(author):
    """
    Clean author names by removing extra spaces and handling multiple authors.
    """
    author = str(author)

    # Remove extra spaces between words
    author = ' '.join(author.split())

    # Split by '|' if there are multiple authors and return as a list
    return author.split('|')

In [ ]:
def get_qid_for_author(author, en_mapper, de_mapper, es_mapper, ja_mapper, fr_mapper):
    """
    Try to find the QID for an author by creating multiple variations of the author's name in multiple languages.
    """
    qid = None

    # Queries for English
    en_queries = [
        author,
        author.replace(' ', '_'),
        author.replace(' ', '_') + '_(novelist)',
        author.replace(' ', '_') + '_(poet)',
        author.replace(' ', '_') + '_(writer)',
        author.replace(' ', '_') + '_(author)',
        author.replace(' ', '_') + '_(journalist)',
        author.replace(' ', '_') + '_(artist)',
    ]

    for query in en_queries:
        qid = en_mapper.title_to_id(query)
        if qid is not None:
            break

    # If QID not found, try German
    if qid is None:
        de_queries = [
            author,
            author.replace(' ', '_'),
            author.replace(' ', '_') + '_(Dichter)',       # poet
            author.replace(' ', '_') + '_(Schriftsteller)', # writer
            author.replace(' ', '_') + '_(Autor)',          # author
            author.replace(' ', '_') + '_(Journalist)',     # journalist
            author.replace(' ', '_') + '_(Autorin)',        # female author
        ]
        for query in de_queries:
            qid = de_mapper.title_to_id(query)
            if qid is not None:
                break

    # If QID not found, try Spanish
    if qid is None:
        es_queries = [
            author,
            author.replace(' ', '_'),
            author.replace(' ', '_') + '_(novelista)',  # novelist
            author.replace(' ', '_') + '_(poeta)',      # poet
            author.replace(' ', '_') + '_(escritor)',   # writer
            author.replace(' ', '_') + '_(autor)',      # author
            author.replace(' ', '_') + '_(periodista)', # journalist
        ]
        for query in es_queries:
            qid = es_mapper.title_to_id(query)
            if qid is not None:
                break

    # If QID not found, try Japanese
    if qid is None:
        ja_queries = [
            author,
            author.replace(' ', '_'),
            author.replace(' ', '_') + '_(小説家)',    # novelist
            author.replace(' ', '_') + '_(詩人)',      # poet
            author.replace(' ', '_') + '_(作家)',      # writer
            author.replace(' ', '_') + '_(著者)',      # author
            author.replace(' ', '_') + '_(ジャーナリスト)', # journalist
        ]
        for query in ja_queries:
            qid = ja_mapper.title_to_id(query)
            if qid is not None:
                break

    # If QID not found, try French
    if qid is None:
        fr_queries = [
            author,
            author.replace(' ', '_'),
            author.replace(' ', '_') + '_(romancier)',   # novelist
            author.replace(' ', '_') + '_(poète)',       # poet
            author.replace(' ', '_') + '_(écrivain)',    # writer
            author.replace(' ', '_') + '_(auteur)',      # author
            author.replace(' ', '_') + '_(journaliste)', # journalist
        ]
        for query in fr_queries:
            qid = fr_mapper.title_to_id(query)
            if qid is not None:
                break

    return qid  # Return QID if found, otherwise None

def map_authors_to_qids(csv_file, output_pickle_file, unique_authors_file, authors_without_qid_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Get unique authors from the 'Book Author' column
    unique_authors = set()

    # Clean author names and add each individual author to the set
    for authors in df['Book Author'].unique():
        cleaned_authors = clean_author_name(str(authors))
        unique_authors.update(cleaned_authors)

    # Initialize a dictionary to store author -> QID mapping
    author_qids = {}

    # Track authors with no QID
    no_qid_authors = []

    # Map each author to a QID
    for author in unique_authors:
        qid = get_qid_for_author(str(author), en_mapper, de_mapper, es_mapper, ja_mapper, fr_mapper)
        if qid is None:
            no_qid_authors.append(author)
            continue
        author_qids[author] = qid


    # Print statistics
    print(f"Total authors: {len(unique_authors)}")
    print(f"Authors without QID: {len(no_qid_authors)}")

    # Save the dictionary to a pickle file
    with open(output_pickle_file, 'wb') as f:
        pickle.dump(author_qids, f)

    print(f"Author-QID mapping saved to {output_pickle_file}")

    # Save all unique author names to a text file
    with open(unique_authors_file, 'w') as f:
        for author in unique_authors:
            f.write(f"{author}\n")

    print(f"Unique authors saved to {unique_authors_file}")

    # Save authors without QID to a text file
    with open(authors_without_qid_file, 'w') as f:
        for author in no_qid_authors:
            f.write(f"{author}\n")

    print(f"Authors without QID saved to {authors_without_qid_file}")

# Example usage
csv_file = '/content/book_summaries.csv'  # Replace with your CSV file
output_pickle_file = 'author_qid_mapping.pickle'  # Output pickle file
unique_authors_file = 'unique_authors.txt'  # Output text file for unique authors
authors_without_qid_file = 'authors_without_qid.txt'  # Output text file for authors without QIDs

map_authors_to_qids(csv_file, output_pickle_file, unique_authors_file, authors_without_qid_file)


Total authors: 6372
Authors without QID: 1982
Author-QID mapping saved to author_qid_mapping.pickle
Unique authors saved to unique_authors.txt
Authors without QID saved to authors_without_qid.txt


Check

In [ ]:
import pandas as pd
import pickle
with open('/content/author_qid_mapping (2).pickle', 'rb') as f:
    author_qid_dict = pickle.load(f)

In [ ]:
def normalize_author_name(name):
    """Normalize the author name by converting to lowercase and removing spaces."""
    return str(name).lower().replace(' ', '')

In [ ]:
with open('/content/author2embedding.pickle', 'rb') as f:
    author2embed = pickle.load(f)
normalized_author2embed = {normalize_author_name(author): embedding for author, embedding in author2embed.items()}

In [ ]:
# Load dataset
df = pd.read_csv('/content/book_summaries.csv')
print('Total samples: ', len(df))

# List to save authors not found in author_qid_dict
not_found_authors = []
count = 0

# Iterate through unique authors in dataset
for authors in df['Book Author'].unique():
    # Clean the author names
    cleaned_authors = clean_author_name(authors)

    # Check if any author is in the author_qid_dict
    if all(normalize_author_name(author) not in normalized_author2embed for author in cleaned_authors):
        count += 1
        not_found_authors.extend(cleaned_authors)

print('Sample without authors embedding: ', count)
print(len(not_found_authors))

Total samples:  13608
Sample without authors embedding:  1409
1510


In [ ]:
with open(authors_without_qid_file, 'w') as f:
        for author in not_found_authors:
            f.write(f"{author}\n")